In [1]:
import requests
from tqdm.notebook import tqdm
import time
import dateparser
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#test_page_addr = "https://obamawhitehouse.archives.gov/the-press-office/2017/01/13/press-briefing-press-secretary-josh-earnest-11317"
#test_page_addrst_page_resp = requests.get(test_page_addr)
#test_page_addrst_page_resp.raise_for_status

In [3]:
TOC_PAGE_ADDR = "https://obamawhitehouse.archives.gov/briefing-room/press-briefings?term_node_tid_depth=36&page=0"
all_links= []
for page_n in tqdm(range(2)):
    c_page_resp = requests.get(TOC_PAGE_ADDR, params={"page":str(page_n)})
    time.sleep(0.3)
    c_page_resp.raise_for_status()
    c_page = c_page_resp.text
    c_soup = BeautifulSoup(c_page, 'html5lib')
    c_soup_vc = c_soup.find("div", {"class" : "view-content"})
    c_soup_rows = c_soup_vc.find_all("div", {"class" : "views-row"})  
    print(c_soup_rows[0].prettify())
    
    for row in c_soup_rows:
        text_date = row.find("span", {"class" : "field-content"}).text 
        text_date = dateparser.parse(text_date)
        text_title = row.find("a").text.strip()
        
        text_link = row.find("a")["href"].strip()
        text_link = "https://obamawhitehouse.archives.gov" + text_link
        
        all_links.append({"date" : text_date,
               "title" : text_title,
               "link" : text_link})
    

  0%|          | 0/2 [00:00<?, ?it/s]

<div class="views-row views-row-1 views-row-odd views-row-first">
 <div class="views-field views-field-created">
  <span class="field-content">
   January 17, 2017
  </span>
 </div>
 <div class="views-field views-field-title">
  <h3 class="field-content">
   <a href="/the-press-office/2017/01/17/press-briefing-press-secretary-josh-earnest-11717">
    Press Briefing by Press Secretary Josh Earnest, 1/17/17
   </a>
  </h3>
 </div>
</div>



C:\Users\user\anaconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


<div class="views-row views-row-1 views-row-odd views-row-first">
 <div class="views-field views-field-created">
  <span class="field-content">
   January 03, 2017
  </span>
 </div>
 <div class="views-field views-field-title">
  <h3 class="field-content">
   <a href="/the-press-office/2017/01/03/press-briefing-press-secretary-josh-earnest-132017">
    Press Briefing by Press Secretary Josh Earnest, 1/3/2017
   </a>
  </h3>
 </div>
</div>



In [4]:
TOC_PAGE_ADDR = "https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks"

all_links = []
for page_n in tqdm(range(5)):
    c_page_resp = requests.get(TOC_PAGE_ADDR, params={"page": str(page_n)})
    time.sleep(0.3)
    c_page_resp.raise_for_status()
    c_page = c_page_resp.text
    c_soup = BeautifulSoup(c_page, "html5lib")
    
    c_soup_vc = c_soup.find("div", {"class" : "view-content"})
    c_soup_rows = c_soup_vc.find_all("div", {"class": "views-row"})
    
    for row in c_soup_rows:
        text_date = row.find("span", {"class" : "field-content"}).text
        text_date = dateparser.parse(text_date)
        
        text_title = row.find("a").text.strip()
        
        text_link = row.find("a")["href"].strip()
        text_link = "https://obamawhitehouse.archives.gov" + text_link
        
        all_links.append({"date": text_date, 
         "title": text_title, 
         "link": text_link})


  0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
all_links[1:13]

[{'date': datetime.datetime(2017, 1, 18, 0, 0),
  'title': 'Remarks by Vice President Joe Biden at the World Economic Forum',
  'link': 'https://obamawhitehouse.archives.gov/the-press-office/2017/01/18/remarks-vice-president-joe-biden-world-economic-forum'},
 {'date': datetime.datetime(2017, 1, 17, 0, 0),
  'title': 'Remarks by Vice President Joe Biden on The Cancer Moonshot',
  'link': 'https://obamawhitehouse.archives.gov/the-press-office/2017/01/17/remarks-vice-president-joe-biden-cancer-moonshot'},
 {'date': datetime.datetime(2017, 1, 17, 0, 0),
  'title': 'Remarks by Vice President Joe Biden With Ukrainian President Petro Poroshenko',
  'link': 'https://obamawhitehouse.archives.gov/the-press-office/2017/01/17/remarks-vice-president-joe-biden-ukrainian-president-petro-poroshenko'},
 {'date': datetime.datetime(2017, 1, 16, 0, 0),
  'title': 'Remarks by the President Honoring the World Series Champion Chicago Cubs',
  'link': 'https://obamawhitehouse.archives.gov/the-press-office/201

In [6]:
all_links_text = []
for link_e in all_links:
    c_page_resp = requests.get(link_e["link"])
    time.sleep(0.3)
    c_page_resp.raise_for_status()
    c_page = c_page_resp.text
    c_soup = BeautifulSoup(c_page, "html5lib")
    
    c_soup_fi = c_soup.find("div", {"class" : "field-items"})
    
    c_soup_par_elems = c_soup_fi.find_all("p")
    c_soup_pars = [pe.text.strip() for pe in c_soup_par_elems]
    c_soup_pars = "\n".join(c_soup_pars).split("\n")
    c_soup_pars = [par.replace("\xa0", " ").replace("  ", " ").strip() for par in c_soup_pars]
    c_soup_pars = [par for par in c_soup_pars if len(par)>0]
    c_soup_all_text = "\n\n".join(c_soup_pars)
    all_links_text.append({**link_e, "text" : c_soup_all_text})
    

In [7]:
all_links_text[2:5]

[{'date': datetime.datetime(2017, 1, 17, 0, 0),
  'title': 'Remarks by Vice President Joe Biden on The Cancer Moonshot',
  'link': 'https://obamawhitehouse.archives.gov/the-press-office/2017/01/17/remarks-vice-president-joe-biden-cancer-moonshot',
  'text': "World Economic Forum\n\nDavos, Switzerland\n\nTHE VICE PRESIDENT: Madam Ambassador, thanks for putting together a crowd for us. (Laughter.)\n\nFolks, I’m delighted to be here and happy to be back with so many distinguished people. And I mean that sincerely. Many of you know as much or more about the subjects I’m going to speak to than I do, including -- is Elizabeth Blackburn here? Elizabeth, a Nobel Laureate, who found the BRCA gene. As that old saying goes, she’s forgotten more about this than I’m going to know.\n\nBut, folks, I’m happy to be back here at the World Economic Forum at Davos to talk about the fight against cancer. And I’m accompanied by Greg Simon, who is Executive Director of administration’s Cancer Moonshot and wh

In [8]:
all_data_df = pd.DataFrame(all_links_text)
all_data_df

,date,title,link,text
0,2017-01-18,Remarks by the President in Final Press Confer...,https://obamawhitehouse.archives.gov/the-press...,James S. Brady Press Briefing Room\n\n2:24 P.M...
1,2017-01-18,Remarks by Vice President Joe Biden at the Wor...,https://obamawhitehouse.archives.gov/the-press...,As Prepared for delivery:\n\nLadies and gentle...
2,2017-01-17,Remarks by Vice President Joe Biden on The Can...,https://obamawhitehouse.archives.gov/the-press...,"World Economic Forum\n\nDavos, Switzerland\n\n..."
3,2017-01-17,Remarks by Vice President Joe Biden With Ukrai...,https://obamawhitehouse.archives.gov/the-press...,"Bankova\n\nKyiv, Ukraine\n\nTHE VICE PRESIDENT..."
4,2017-01-16,Remarks by the President Honoring the World Se...,https://obamawhitehouse.archives.gov/the-press...,East Room\n\n1:40 P.M. EST\n\nTHE PRESIDENT: T...
5,2017-01-13,Weekly Address: The Honor of Serving You as Pr...,https://obamawhitehouse.archives.gov/the-press...,"WASHINGTON, DC — This week, President Obama de..."
6,2017-01-12,Remarks by the President and the Vice Presiden...,https://obamawhitehouse.archives.gov/the-press...,State Dining Room\n\n3:50 P.M. EST\n\nTHE PRES...
7,2017-01-12,Remarks by the Vice President on Nuclear Security,https://obamawhitehouse.archives.gov/the-press...,Remarks by the Vice President on Nuclear Secur...
8,2017-01-12,Remarks by the Vice President on Nuclear Security,https://obamawhitehouse.archives.gov/the-press...,Remarks by the Vice President on Nuclear Secur...
9,2017-01-10,Remarks by the President in Farewell Address,https://obamawhitehouse.archives.gov/the-press...,"McCormick Place\n\nChicago, Illinois\n\n8:02 P..."
